In [31]:
import json
import urllib.request
from io import StringIO
from html.parser import HTMLParser
import pandas as pd

In [32]:
class MLStripper(HTMLParser): #Strips HTML from our downloaded text.  From stackoverflow
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()
def strip_tags(html): #Strips HTML from our downloaded text.  From stackoverflow
    s = MLStripper()
    s.feed(html)
    return s.get_data()

#initialize needed variables
policy_list = []
policy_dict = {}

#make our subset to flag used applications for MSDS as we come across them
MSDS_use = ['Slack','Kaltura','Microsoft','Google','Youtube','Trello']
MSDS_use_flag = 0

#these data points are sometimes empty, override NoneType error
#with safety net defaults
privacy_stmt_count = 0
policy_text = None

#real deal starting point
#cur_pg = 'https://api.usableprivacy.org/websites/'

#movable testing starting point for faster runtime
cur_pg = 'https://api.usableprivacy.org/websites/?page=1195'

#while loop will continue as long as the page has a 'next' value
while cur_pg:  
    #pull in our page
    load = json.load(urllib.request.urlopen(cur_pg))
    
    #we have these 6 indicies as a pattern on all our API pages
    for i in [0,1,2,3,4,5]:
        #use try loop to make sure we keep going and just print out trouble spots in the except
        try:
            #code MSDS flag as 1 if we've got something
            if (load['results'][i]['title'] in MSDS_use):
                MSDS_use_flag = 1
            
            #default to 0 if no privacy statement count if empty
            if not(load['results'][i]['num_privacy_statements'][0]):
                privacy_stmt_count = 0
            else:
                privacy_stmt_count = load['results'][i]['num_privacy_statements'][0]
            


            if not(load["results"][i]["fine_grained_policy"]) and load["results"][i]["scraped_policies"][0]["segments"]:
                j = 0
                policy_text = ""
                while load["results"][i]["scraped_policies"][0]["segments"][j]["text"]:
                    
                    try:
                        n = j+1
                        load["results"][i]["scraped_policies"][0]["segments"][n]["text"]
                        j += 1
                    except:
                        break

            #default to 'No policy text available' if empty
            elif not(load["results"][i]["fine_grained_policy"]):
                policy_text = 'No policy text available'
            
            
            
            
            #default to 'No policy text available' if empty
            # if not(load["results"][i]["fine_grained_policy"]):
            #     policy_text = 'No policy text available'
            else:
                policy_text = strip_tags(load['results'][i]["fine_grained_policy"]["text"])
                policy_text = policy_text.replace("|", "")
                policy_text = " ".join(policy_text.split()) #strip all the extra white space
            
            #remove any data points we can't use - unusable word counts
            if load['results'][i]['policy_word_count'] > 0:
            #fill a holder dictionary with all the needed info
                policy_dict = {'Site URL':load['results'][i]['url'], 'Site Title':load['results'][i]['title'],'Policy Word Count':load['results'][i]['policy_word_count'], 'Privacy Statements Count':privacy_stmt_count, 'MSDS Use Flag':MSDS_use_flag, 'API URL':cur_pg, 'Policy Text':policy_text}
            
            
            #add the dictionary to our list
            policy_list.append(policy_dict)
        
        
        #let the loop keep going and just print out where failures occur
        except:
            print("error at " + cur_pg + "  index " + str(i))
        
            #reset our flag for next result
            MSDS_use_flag = 0
    #after looping through our results, get the next page. If empty, this breaks our while
    cur_pg = load['next']

error at https://api.usableprivacy.org/websites/?page=1195  index 4
error at https://api.usableprivacy.org/websites/?page=1195  index 5


In [34]:
#with our complete list, convert it into a dataframe
policy_df = pd.DataFrame(policy_list)
#dump into a csv
#policy_df.to_csv('privacy policies.csv')

#take a look at the data, print out first five rows of dataframe
policy_df.head()

# #my Spyder won't give me all the columns (just returns a ... inbetween the first and last) so I print out subsets here
# policy_df.iloc[:,0:2].tail()
# policy_df.iloc[:,2:4].tail()
# policy_df.iloc[:,4:6].tail()

,Site URL,Site Title,Policy Word Count,Privacy Statements Count,MSDS Use Flag,API URL,Policy Text
0,kano.me,kano.me,2810,24,0,https://api.usableprivacy.org/websites/?page=1195,
1,petcube.com,Petcube,6379,62,0,https://api.usableprivacy.org/websites/?page=1195,
2,www2.meethue.com,Meethue,3021,26,0,https://api.usableprivacy.org/websites/?page=1195,
3,www.runtastic.com,Runtastic,5790,52,0,https://api.usableprivacy.org/websites/?page=1195,


In [ ]:

#print out last five rows of data frame
policy_df.tail()

In [61]:
#store some basic global properties of our dataframe
total_count = len(policy_df)

In [43]:
#now with all our data downloaded, we clean up and do some data transformations

#create new calculated column for the estimated read time based on average reading words per minute
policy_df['Read Time(min)'] = round(policy_df['Policy Word Count']/240,2) #average wpm is 240

#check out new column
policy_df.head()

,Site URL,Site Title,Policy Word Count,Privacy Statements Count,MSDS Use Flag,API URL,Policy Text,Read Time(min)
0,kano.me,kano.me,2810,24,0,https://api.usableprivacy.org/websites/?page=1195,,11.71
1,petcube.com,Petcube,6379,62,0,https://api.usableprivacy.org/websites/?page=1195,,26.58
2,www2.meethue.com,Meethue,3021,26,0,https://api.usableprivacy.org/websites/?page=1195,,12.59
3,www.runtastic.com,Runtastic,5790,52,0,https://api.usableprivacy.org/websites/?page=1195,,24.12


In [63]:
#how many policies take longer than 10 minutes to read
len(policy_df[(policy_df['Read Time(min)']>10)])

#how many policies take longer than 10 minutes to read
len(policy_df[(policy_df['Read Time(min)']>30)])

#how many policies take longer than 10 minutes to read
len(policy_df[(policy_df['Read Time(min)']>145)])

0

In [49]:
#which policies are required for our MSDS program
policy_df[(policy_df['MSDS Use Flag']==1)]


,Site URL,Site Title,Policy Word Count,Privacy Statements Count,MSDS Use Flag,API URL,Policy Text,Read Time(min)


In [54]:
#how many minutes would it take to read all the policies required for the MSDS program
sum(policy_df[(policy_df['MSDS Use Flag']==1)]['Read Time(min)'])

0

In [64]:
time_limit = float(input("How much time are you willing to spend reading a privacy policy? Input a time limit in minutes and find out the percentage of policies exceeding it! Input 0 to skip"))
if time_limit > 0:
    policy_count = len(policy_df[(policy_df['Read Time(min)']>time_limit)])
    print("There are {} policies which take the average person at least {} minutes to read".format(policy_count,time_limit))
    print("The percentage of all policies exceeding this time limit is {}%.".format(policy_count/total_count))

There are 4 policies which take the average person at least 5.0 minutes to read
The percentage of policies exceeding this time limit is 1.0%.
